
#  高性能プログラミングと性能測定(3) --- SIMDプログラミング


# 1. 概要
* ベクタ型と(必要ならば)intrinsicsを使って, 確実にSIMD化する手法を学ぶ
* コンパイラの-Sオプションで実際にSIMD化出来ていることを確かめる
* そのもとで性能を計測する



# 2. SIMD化が(容易に)可能なコード
 * SIMDはSingle Instruction Multiple Dataの略
 * SIMD命令は, ひとつの命令で複数 (Oakbridge CXのCPU (Cascade Lake)ではfloat, intなど32 bitの要素であれば16個, double, longなど64 bitの要素であれば8個)の要素に対する演算を施す命令
 * したがってSIMD化できるための前提条件は, 複数データに「同じ演算を施す」ということ
 * それもそのような演算がある程度の数連続して行わるようでないと, SIMD命令を施すためのデータの配置(SIMDレジスタへ格納するなど)のオーバーヘッドがすぐに大きくなってしまう
 * したがって, SIMD化出来るコードの近似としては, 「単純なループであって, ループ本体(繰り返しの一回分)は, ほとんど同じ動作をするもの」と理解しておけば良い



# 3. SIMD化が(容易に)可能なコード
 * 以下の積分計算

$$ \int_a^b x^2\, dx $$

のSIMD化を試みる. いわゆる区分求積法を用いた積分の計算.

 * 答えがわかりきっていてやる気がおきないかもしれないがSIMDを説明するのに最も単純な例ということで勘弁していただきたい.
 * まずは通常の(SIMD化されていない)コード
 

In [ ]:
%%writefile int_x2.c
#include <stdio.h>
#include <stdlib.h>

double int_x2(double a, double b, long n) {
  double s = 0.0;
  double x = a;
  double dx = (b - a) / (double)n;
  for (long i = 0; i < n; i++) {
    s += x * x;
    x += dx;
  }
  return s * dx;
}

int main(int argc, char ** argv) {
  double a = (argc > 1 ? atof(argv[1]) : 0.0);
  double b = (argc > 2 ? atof(argv[2]) : 1.0);
  long n   = (argc > 3 ? atof(argv[3]) : 1000L * 1000L * 1000L);
  double s = int_x2(a, b, n);
  printf("s = %f\n", s);
  return 0;
}

In [ ]:
gcc -O3 -march=native int_x2.c -o int_x2

In [ ]:
./int_x2


 * このコードでもっとも時間がかかるのは明らかに以下のループ(それ以外ほとんど何もしていないのだから当然. ちゃんと確かめたければ実行時間がnにほぼ比例していることを確かめれば良い)

```
  for (long i = 0; i < n; i++) {
    s += x * x;
    x += dx;
  }
```

 * SIMD化の目標はこのループの複数(Cascade Lakeでは8つ)の繰り返しを同時に処理することである

 * そう思ってループの繰り返し本体を眺めると, 基本的にどの繰り返しでも同じ動作をしていて, 違いはxに入っている値が違うだけとわかる

 * 擬似的に書けば以下のような処理をすれば, 4回分まとめて処理(SIMD化)ができそうとわかる

```
  // 連続する8回の繰り返しをまとめて処理
  for (long i = 0; i < n; i += 4) {
    s += { x, x+dx, x+2*dx, ..., x+7*dx } * { x, x+dx, x+2*dx, ..., x+7*dx };
    x += 8 * dx;
  }
```
 * なお, nは8で割り切れると仮定する(割り切れない場合は, 端数部分を別途処理する必要があるが普通は近くの8の倍数に切り上げてしまえば計算の目的は達成できるのでここでは気にしないことにする).

 * きちんと動くコードにしたものが以下


In [ ]:
%%writefile int_x2_simd.c
#include <stdio.h>
#include <stdlib.h>

/* doubleを8つ並べたデータ型(doublev) */
typedef double doublev __attribute__((vector_size(64),aligned(sizeof(double))));
enum { n_lanes = sizeof(doublev) / sizeof(double) };

double int_x2_simd(double a, double b, long n) {
  doublev s = {0,0,0,0,0,0,0,0};
  // n をレーン数の倍数に
  n += n_lanes - 1;
  n -= n % n_lanes;
  double dx = (b - a) / (double)n;
  doublev x = {a,a+dx,a+2*dx,a+3*dx,a+4*dx,a+5*dx,a+6*dx,a+7*dx};
  for (long i = 0; i < n; i += n_lanes) {
    s += x * x;
    x += n_lanes * dx;
  }
  return (s[0] + s[1] + s[2] + s[3] + s[4] + s[5] + s[6] + s[7]) * dx;
}

int main(int argc, char ** argv) {
  double a = (argc > 1 ? atof(argv[1]) : 0.0);
  double b = (argc > 2 ? atof(argv[2]) : 1.0);
  long n   = (argc > 3 ? atof(argv[3]) : 1000L * 1000L * 1000L);
  double s = int_x2_simd(a, b, n);
  printf("s = %f\n", s);
  return 0;
}

In [ ]:
gcc -O3 -march=native int_x2_simd.c -o int_x2_simd

In [ ]:
./int_x2_simd

## 3-1. timeコマンドでお手軽に経過時間比較

In [ ]:
time ./int_x2

In [ ]:
time ./int_x2_simd

* int_x2 と int_simd の実行時間を比べてみよ
* より確信を深めるために命令数を測ってみる
* 以下がそのためのコマンドであるがOakbridge CX上でないと結果は得られない

<font color="blue">on Oakbridge CX</font> 

In [ ]:
perf stat ./int_x2

<font color="blue">on Oakbridge CX</font> 

In [ ]:
perf stat ./int_x2_simd

* 実行命令数が, ほぼぴったり n (= 1000,000,000) の5倍, int_x2_simdではそれがほぼその 1/8 となっていることから, SIMD化はできていると判断できる



* 生成されたアセンブリ言語を見てみよう
* 結果を短くするため, 肝心の関数だけを単体でコンパイルする


In [ ]:
%%writefile int_x2_simd_s.c
/* doubleを8つ並べたデータ型(doublev) */
typedef double doublev __attribute__((vector_size(64),aligned(sizeof(double))));
enum { n_lanes = sizeof(doublev) / sizeof(double) };

double int_x2_simd(double a, double b, long n) {
  doublev s = {0,0,0,0,0,0,0,0};
  // n をレーン数の倍数に
  n += n_lanes - 1;
  n -= n % n_lanes;
  double dx = (b - a) / (double)n;
  doublev x = {a,a+dx,a+2*dx,a+3*dx,a+4*dx,a+5*dx,a+6*dx,a+7*dx};
  for (long i = 0; i < n; i += n_lanes) {
    s += x * x;
    x += n_lanes * dx;
  }
  return (s[0] + s[1] + s[2] + s[3] + s[4] + s[5] + s[6] + s[7]) * dx;
}

In [ ]:
gcc -O3 -march=native int_x2_simd.c -S
cat int_x2_simd.s


* 繰り返されている部分は以下

```
.L3:
        addq    $8, %rdx
        vfmadd231pd     %zmm2, %zmm2, %zmm3
        vaddpd  %zmm0, %zmm2, %zmm2
        cmpq    %rax, %rdx
        jne     .L3
```

* 繰り返し1回あたりの命令数が5であると確認できた



* cycles 数は, nのほぼ4倍で, それはこの繰り返しが一回あたり 4 cyclesかかることを示している
* それはなぜか? 
 * 答えは, s += x * x; (s = x * x + s)に対応する命令である, vfmaddpd の遅延が4だからである
 * このvfmaddpd 命令は, ある繰り返しで計算されてsに格納された結果を次の繰り返しで使っている(依存関係がある)ので, 4 cycles/命令 のペースでしか実行できない
* 今は天下り的に vfmaddpd に注目したが, もう少しきちんとした考察は同様のことをすべての命令に対して行い, 依存関係を元に, 一周するのにかかる時間を計算することである. 
* x += 8 * dx (x = x + 8 * dx) に対して生成された vaddpd 命令(8 * dx の掛け算はループ内で行う必要がないのでループの外側で一度だけ8 * dxが計算されている. そのためループ内での掛け算は必要なくなり, 足し算だけになっている)間でも, 依存関係が生じている. その遅延は3である. したがってループ全体の1周あたりの遅延は5となる
* なお鋭い人はなぜ 5 + 3 = 8 とならず, 5 になるのかと思うかも知れない
* その理由は, s = x * x + s の計算と, x = x + 8 * dx の計算の間に依存関係がないためで, 両者はプロセッサの中で並行して計算されている(スーパースカラー)



* 上記のコードを見てvector_size(64)とか, 4 みたいな定数が各所に出てきて汚いと思った人は正しい感覚の持ち主
* コンパイラが勝手に最適化してくれないものをしようとしているのである程度汚くなるのは避けられないが, それにしても vector_size が変わっても一箇所だけ書き換えれば済むようにしておくのはいい心がけである
* また, 一箇所書き換えれば floatも doubleも扱えるようにするのも良い心がけ
* それをやったのが以下


In [ ]:
%%writefile int_x2_simd_v.c
#include <stdio.h>
#include <stdlib.h>

/* double/float をこれだけで変更できるように */
typedef double real;

/* realを32バイト分 (float x 8 または double x 4)並べたデータ型 */
typedef real realv __attribute__((vector_size(64),aligned(sizeof(real))));

/* number of lanes (1 SIMD変数あたりの要素数) */
enum { nl = sizeof(realv) / sizeof(real) };

real int_x2_simd_v(real a, real b, long n) {
  real dx = (b - a) / (real)n;
  realv s;
  realv x;
  /* sの全要素を0に */
  for (long i = 0; i < nl; i++) {
    s[i] = 0.0;
  }
  /* x = { a, a+dx, a+2*dx, ... } */
  for (long i = 0; i < nl; i++) {
    x[i] = a + i * dx;
  }
  /* 本題 */
  for (long i = 0; i < n; i += nl) {
    s += x * x;
    x += nl * dx;
  }
  /* sの各要素に入った部分和を足し合わせる */
  real ss = 0.0;
  for (long i = 0; i < nl; i++) {
    ss += s[i];
  }
  return ss  * dx;
}

int main(int argc, char ** argv) {
  real a = (argc > 1 ? atof(argv[1]) : 0.0);
  real b = (argc > 2 ? atof(argv[2]) : 1.0);
  long n   = (argc > 3 ? atof(argv[3]) : 1000L * 1000L * 1000L);
  real s = int_x2_simd_v(a, b, n);
  printf("s = %f\n", s);
  return 0;
}

In [ ]:
gcc -O3 -march=native int_x2_simd_v.c -o int_x2_simd_v

In [ ]:
time ./int_x2_simd_v


# <font color="green"> Problem 1 :  積分計算のSIMD化</font>
* 
$$ \int_a^b \frac{dx}{1+x^2} $$
を計算する以下のコードを実行して実行時間を観察せよ (Shift + Enterを叩くだけ)
* その下のセル(同じコードがコピーされている)を修正して, SIMD化せよ
* gcc -S オプションを使って, どのような命令が使われているか確かめよ
* SIMD化されているものとそうでないもので, 実行時間を比べよ
* -S オプションを用いて, どのような命令がループで繰り返されているか調べよ. 1+x*x や その逆数に相当する命令はどれか?
* Oakbridge CXでperfコマンドを用いてcyclesを測るか, このページ上で_rdtsc関数を用いてreference cyclesを測るなどして, ループ一回あたりのサイクル数を求めよ
* なぜそのサイクル数になるのか? ヒント: この計算では割り算を行っており, 割り算命令は遅延も大きいし, クロックあたりに実行可能な最大命令数(スループット)も非常に少ない. https://software.intel.com/sites/landingpage/IntrinsicsGuide/ で調べてみよ


In [ ]:
%%writefile int_inv_1_x2.c
#include <stdio.h>
#include <stdlib.h>

double int_inv_1_x2(double a, double b, long n) {
  double s = 0.0;
  double x = a;
  double dx = (b - a) / (double)n;
  for (long i = 0; i < n; i++) {
    s += 1 / (1 + x * x);
    x += dx;
  }
  return s * dx;
}

int main(int argc, char ** argv) {
  double a = (argc > 1 ? atof(argv[1]) : 0.0);
  double b = (argc > 2 ? atof(argv[2]) : 1.0);
  long n   = (argc > 3 ? atof(argv[3]) : 1000L * 1000L * 1000L);
  double s = int_inv_1_x2(a, b, n);
  printf("s = %f\n", s);
  return 0;
}

In [ ]:
gcc -O3 -march=native int_inv_1_x2.c -o int_inv_1_x2

In [ ]:
time ./int_inv_1_x2


* 以下を修正してSIMD化せよ

In [ ]:
%%writefile int_inv_1_x2_a.c
#include <stdio.h>
#include <stdlib.h>

double int_inv_1_x2(double a, double b, long n) {
  double s = 0.0;
  double x = a;
  double dx = (b - a) / (double)n;
  for (long i = 0; i < n; i++) {
    s += 1 / (1 + x * x);
    x += dx;
  }
  return s * dx;
}

int main(int argc, char ** argv) {
  double a = (argc > 1 ? atof(argv[1]) : 0.0);
  double b = (argc > 2 ? atof(argv[2]) : 1.0);
  long n   = (argc > 3 ? atof(argv[3]) : 1000L * 1000L * 1000L);
  double s = int_inv_1_x2(a, b, n);
  printf("s = %f\n", s);
  return 0;
}

In [ ]:
gcc -O3 -march=native int_inv_1_x2_a.c -o int_inv_1_x2_a

In [ ]:
time ./int_inv_1_x2_a